# Second Project: MACD and Portfolio Check

## Introduction
This project focuses on analyzing the Moving Average Convergence Divergence (MACD) indicator and checking portfolio performance. The aim is to explore the effectiveness of the MACD indicator in trading strategies and assess the performance of a given portfolio.

## Table of Contents
1. Introduction
2. MACD Analysis
3. Portfolio Performance Check
4. Conclusion
5. References

# **MACD**

### **October 30 2022**

We thought that a MACD-graph could not not help, so we decided to implement it as well. I remember that my coding at this time was heavily influenced by the "Je-suit-TM"-profile on Github. Thank you!

Here is a translated note from our log:  "*Tried to find models for indicators such as MACD. Since I couldn't find any that solve it in an elegant manner, the only choice is to do it myself.*"

Notable is the rapid developement of the programming over the last month. Let's go through the portfolio function since it's quite interesting:

#  Portfolio Function

The name 'portfolio' almost says it all. We want to calculate a hypothetical portfolio over time and we want to do it fast, so we use pandas. Below are all the necessary steps.

## Function Inputs and Parameters

- `signals`: DataFrame containing trading signals and closing prices.
  - \( S \) - Trading signals indicating buy (1) or sell (-1) positions.
  - \( P \) - Closing prices of the asset.
- `capital0`: Initial capital, defaulting to $10,000.
- `positions`: Number of shares involved per trading position, defaulting to 100.

## Mathematical Formulation

### Step 1: Cumulative Sum of Trading Signals

$$\text{cumsum}_i = \sum_{j=1}^i S_j $$

- This alculates the cumulative sum of signals. Basically we have a series of zeros with 1 or -1 here and there and we want to change that to a series of only 1 or -1. This sum does exactly that. Why do we want that? In the near future, we would like to add some kind of return to the series and this is just purely a better format for that.  

### Step 2: Holdings Calculation

$$\text{holdings}_i = \text{cumsum}_i \cdot P_i \cdot \text{positions} $$

- This computes the value of current holdings based on net positions and current price. So we take i.e. close price and add that to the series.

### Step 3: Cash Calculation

$$\text{cash}_i = \text{capital}_0 - \sum_{j=1}^i (S_j \cdot P_j \cdot \text{positions}) $$

- Calculates the remaining cash after accounting for all trading transactions up to time \( i \).

### Step 4: Total Asset Calculation

$$ \text{total asset}_i = \text{holdings}_i + \text{cash}_i $$

- Sum of current holdings and remaining cash, representing total value of the asset portfolio.

### Step 5: Portfolio Returns Calculation

$$ \text{return}_i = \frac{\text{total asset}_i - \text{total asset}_{i-1}}{\text{total asset}_{i-1}} $$

- Calculates the percentage change in the portfolio's total value from the previous time step to the current one.

### Step 6: Total Return Calculation

$$ \text{total return}_i = \prod_{j=1}^i (1 + \text{return}_j) $$

- Computes the cumulative product of returns, indicating the overall growth of the portfolio over time. We use cumprod since we want compounded returns. Another way we could have done this is taking the logarithm of the data and then just use cumsum (multiplication becomes addition when we log the series).

## Outputs

- **Portfolio DataFrame**: Returns a DataFrame structured as follows:
  - `holdings`: Value of holdings over time.
  - `cash`: Remaining cash over time.
  - `total asset`: Total asset value over time.
  - `return`: Percentage change in total asset value.
  - `total return`: Cumulative growth of the portfolio.
  - `signals`: Trading signals used in computations.
  - `date`: Dates for each data point.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import yfinance as yf

In [ ]:
ma1=12
ma2=26
slicer=5
initial_wealth=1000

In [ ]:
# This is just the signal creation, taken from JSTM
def macd_signal_generation(df,method):

    signals=method(df)
    signals['positions']=0

    #positions becomes and stays one once the short moving average is above long moving average
    signals['positions'][ma1:]=np.where(signals['ma1'][ma1:]>=signals['ma2'][ma1:],1,0)


    #we take the difference to generate real trade signal
    signals['signals']=signals['positions'].diff()

    #oscillator is the difference between two moving average
    #when it is positive, we long, vice versa
    signals['oscillator']=signals['ma1']-signals['ma2']

    return signals

In [ ]:
def portfolio(signals,capital0=10000,positions=100):

    #cumsum column is created to check the holding of the position
    signals['cumsum']=signals['signals'].cumsum()


    portfolio=pd.DataFrame()
    portfolio['holdings']=signals['cumsum']*signals['Close']*positions
    portfolio['cash']=capital0-(signals['signals']*signals['Close']*positions).cumsum()
    portfolio['total asset']=portfolio['holdings']+portfolio['cash']
    portfolio['return']=portfolio['total asset'].pct_change()
    portfolio['total return']=(1+portfolio['return']).cumprod()
    portfolio['signals']=signals['signals']
    portfolio['date']=signals['Date']
    portfolio.set_index('date',inplace=True)
    portfolio[['holdings', 'cash','total asset']].to_csv('signals_data_1.csv')
    print(portfolio['total return'])
    #if you want to add a short strategy, do the exact same thing and compare the total return percentage: (1+return_long+return_short).cumprod()
    return portfolio


In [ ]:
def profit(portfolio):

    fig=plt.figure()
    bx=fig.add_subplot(111)

    portfolio['total asset'].plot(label='Total Asset')

    #long/short position markers related to the portfolio
    #the same mechanism as the previous one
    #replace close price with total asset value
    bx.plot(portfolio['signals'].loc[portfolio['signals']==1].index,portfolio['total asset'][portfolio['signals']==1],lw=0,marker='^',c='g',label='long')
    bx.plot(portfolio['signals'].loc[portfolio['signals']<0].index,portfolio['total asset'][portfolio['signals']<0],lw=0,marker='v',c='r',label='short')

    plt.legend(loc='best')
    plt.grid(True)
    plt.xlabel('Date')
    plt.ylabel('Asset Value')
    plt.title('Total Asset')
    plt.show()


In [ ]:
#plotting the backtesting result
def macd_plot(new, ticker):


    fig=plt.figure(figsize=[12,8])
    cx=fig.add_subplot(211)

    new['oscillator'].plot(kind='bar',color='r')

    plt.legend(loc='best')
    plt.grid(True)
    plt.xticks([])
    plt.xlabel('')
    plt.title('MACD Oscillator')

    bx=fig.add_subplot(212)

    new['ma1'].plot(label='ma1')
    new['ma2'].plot(label='ma2',linestyle=':')

    plt.legend(loc='best')
    plt.grid(True)
    plt.title('MA for {}'.format(ticker))
    plt.savefig('MACD_MA_graph.png')
